<a href="https://colab.research.google.com/github/jalevano/tfm_uoc_datascience/blob/main/03_YOLO_Visor_Mascaras.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
# =============================================================================
# YOLOV8-SEG VISUALIZADOR AVANZADO DE MÁSCARAS
# =============================================================================
# Proyecto: Visualización y Análisis de Resultados YOLOv8-seg
# Funcionalidad: Carga NPZ + JSON, genera visualizaciones avanzadas
# Compatible con: Resultados del 02_YOLOv8_Seg_Evaluador.py
# =============================================================================

# INSTALACIÓN DE DEPENDENCIAS
# =============================================================================

# !pip install -q opencv-python matplotlib Pillow
# !pip install -q numpy scipy
# !pip install -q seaborn plotly
# !pip install -q tqdm

print("✅ Dependencias instaladas correctamente")

✅ Dependencias instaladas correctamente


In [2]:
# =============================================================================
# IMPORTACIONES Y MONTAJE DE DRIVE
# =============================================================================

import os
import sys
import json
from pathlib import Path
from typing import List, Dict, Any, Optional, Tuple
from dataclasses import dataclass
import warnings

import numpy as np
import cv2
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.patches as mpatches
from matplotlib.gridspec import GridSpec
import seaborn as sns
from tqdm.auto import tqdm

from google.colab import drive

# Configuración
warnings.filterwarnings('ignore')
plt.rcParams['figure.dpi'] = 100
plt.rcParams['savefig.dpi'] = 150
plt.rcParams['font.size'] = 10

# Montar Drive
drive_path = Path("/content/drive/MyDrive")
if not drive_path.exists():
    print("📁 Montando Google Drive...")
    drive.mount('/content/drive')
    print("✅ Drive montado correctamente")
else:
    print("✅ Google Drive ya está montado")

print("✅ Configuración completada")

📁 Montando Google Drive...
Mounted at /content/drive
✅ Drive montado correctamente
✅ Configuración completada


In [9]:
# =============================================================================
# CONFIGURACIÓN DEL VISUALIZADOR
# =============================================================================

@dataclass
class ConfigVisualizadorYOLO:
    """Configuración centralizada del visualizador YOLOv8"""

    # Rutas base - ⚠️ MODIFICAR SEGÚN TU CONFIGURACIÓN
    BASE_PATH: Path = Path("/content/drive/MyDrive/TFM/yolov8_seg")
    RESULTADOS_PATH: Path = BASE_PATH / "resultados"
    IMAGENES_PATH: Path = Path("/content/drive/MyDrive/TFM/yolov8_seg/imagenes")
    OUTPUT_PATH: Path = BASE_PATH / "visualizaciones_avanzadas"

    # Opciones de visualización
    GENERAR_MASCARAS_INDIVIDUALES: bool = True
    GENERAR_OVERLAYS: bool = True
    GENERAR_RECORTES: bool = True
    GENERAR_ALPHA: bool = True
    GENERAR_COMPARATIVAS: bool = True
    GENERAR_ESTADISTICAS: bool = True

    # Colores para visualización por modelo (RGB)
    COLORES_MODELOS = {
        'yv8n_seg': (255, 0, 0),      # Rojo - Nano
        'yv8s_seg': (0, 255, 0),      # Verde - Small
        'yv8m_seg': (0, 0, 255),      # Azul - Medium
        'yv8l_seg': (255, 255, 0),    # Amarillo - Large
        'yv8x_seg': (255, 0, 255)     # Magenta - XLarge
    }

    # Transparencia para overlays
    ALPHA_OVERLAY: float = 0.5

    def __post_init__(self):
        """Crea directorios necesarios"""
        self.OUTPUT_PATH.mkdir(parents=True, exist_ok=True)
        (self.OUTPUT_PATH / "por_imagen").mkdir(exist_ok=True)
        (self.OUTPUT_PATH / "comparativas").mkdir(exist_ok=True)
        (self.OUTPUT_PATH / "estadisticas").mkdir(exist_ok=True)

# Crear instancia global
config = ConfigVisualizadorYOLO()
print(f"✅ Configuración creada")
print(f"📁 Salida: {config.OUTPUT_PATH}")

✅ Configuración creada
📁 Salida: /content/drive/MyDrive/TFM/yolov8_seg/visualizaciones_avanzadas


In [4]:
# =============================================================================
# CARGADOR DE RESULTADOS YOLOV8
# =============================================================================

class CargadorResultadosYOLO:
    """Carga y organiza resultados de evaluaciones YOLOv8-seg"""

    def __init__(self, directorio_ejecucion: Path):
        self.directorio_ejecucion = Path(directorio_ejecucion)
        self.datos_cargados = {}

    def cargar_ejecucion_completa(self) -> Dict[str, Any]:
        print(f"\n{'='*80}")
        print(f"CARGANDO RESULTADOS DE: {self.directorio_ejecucion.name}")
        print(f"{'='*80}\n")

        # Buscar directorios de modelos
        modelos_dirs = [d for d in self.directorio_ejecucion.iterdir()
                       if d.is_dir() and d.name.startswith('yv8')]

        resultados = {}

        for modelo_dir in modelos_dirs:
            modelo_nombre = modelo_dir.name
            print(f"📂 Cargando modelo: {modelo_nombre}")

            # Cargar JSONs
            json_files = list(modelo_dir.glob("*.json"))
            if not json_files:
                print(f"   ❌ No se encontraron archivos JSON")
                continue

            # Cargar todos los JSONs del modelo
            imagenes_data = {}

            for json_file in json_files:
                with open(json_file, 'r') as f:
                    json_data = json.load(f)

                # Procesar cada resultado
                for resultado in json_data.get('resultados', []):
                    nombre_imagen = resultado['nombre_imagen']

                    # Buscar archivo de máscaras
                    archivo_mascaras = resultado.get('archivo_mascaras')

                    if archivo_mascaras:
                        # Ruta completa al NPZ
                        npz_path = self.directorio_ejecucion / archivo_mascaras

                        if npz_path.exists():
                            # Cargar máscaras NPZ
                            mask_data = np.load(npz_path, allow_pickle=True)
                            metadatos = mask_data['metadatos'].item()

                            # Extraer máscaras por umbral
                            mascaras_por_umbral = {}

                            for key in mask_data.files:
                                if key.startswith('umbral_') and key != 'metadatos':
                                    # Extraer información del umbral
                                    parts = key.split('_')
                                    if len(parts) >= 4:
                                        umbral_key = f"{parts[0]}_{parts[1]}"
                                        mask_idx = int(parts[3])

                                        if umbral_key not in mascaras_por_umbral:
                                            mascaras_por_umbral[umbral_key] = []

                                        mascaras_por_umbral[umbral_key].append({
                                            'mascara': mask_data[key],
                                            'idx': mask_idx
                                        })

                            # Agregar a imagenes_data
                            if nombre_imagen not in imagenes_data:
                                imagenes_data[nombre_imagen] = {
                                    'json_data': resultado,
                                    'mascaras_npz': [],
                                    'config': json_file.stem
                                }

                            imagenes_data[nombre_imagen]['mascaras_npz'].append({
                                'metadatos': metadatos,
                                'mascaras_por_umbral': mascaras_por_umbral,
                                'archivo': npz_path
                            })

            resultados[modelo_nombre] = {
                'metadatos': json_data.get('metadata', {}),
                'imagenes': imagenes_data
            }

            num_imagenes = len(imagenes_data)
            total_mascaras = sum(
                len(npz['mascaras_por_umbral'])
                for img in imagenes_data.values()
                for npz in img['mascaras_npz']
            )
            print(f"   ✅ {num_imagenes} imágenes, {total_mascaras} grupos de máscaras")

        self.datos_cargados = resultados

        print(f"\n{'='*80}")
        print(f"✅ CARGA COMPLETADA")
        print(f"   Modelos cargados: {len(resultados)}")
        print(f"{'='*80}\n")

        return resultados

    def obtener_lista_imagenes(self) -> List[str]:
        """Obtiene lista de nombres de imágenes procesadas"""
        imagenes = set()
        for modelo_data in self.datos_cargados.values():
            imagenes.update(modelo_data['imagenes'].keys())
        return sorted(list(imagenes))

    def obtener_modelos_disponibles(self) -> List[str]:
        """Obtiene lista de modelos cargados"""
        return list(self.datos_cargados.keys())

print("✅ Clase CargadorResultadosYOLO definida")


✅ Clase CargadorResultadosYOLO definida


In [5]:
# =============================================================================
# GENERADOR DE MÁSCARAS INDIVIDUALES
# =============================================================================

class GeneradorMascarasIndividualesYOLO:
    """Genera visualizaciones de máscaras individuales YOLOv8"""

    def __init__(self, config: ConfigVisualizadorYOLO):
        self.config = config

    def generar_para_imagen(self, nombre_imagen: str,
                           datos_modelos: Dict,
                           ruta_imagen_original: Path) -> Dict[str, List[Path]]:
        """Genera todas las máscaras individuales para una imagen"""

        # Crear directorio para esta imagen
        dir_imagen = self.config.OUTPUT_PATH / "por_imagen" / Path(nombre_imagen).stem
        dir_imagen.mkdir(parents=True, exist_ok=True)
        dir_personas = dir_imagen / "personas_individuales"
        dir_personas.mkdir(exist_ok=True)

        # Cargar imagen original
        imagen_original = cv2.imread(str(ruta_imagen_original))
        if imagen_original is None:
            print(f"❌ No se pudo cargar: {ruta_imagen_original}")
            return {}

        imagen_original = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB)
        h_orig, w_orig = imagen_original.shape[:2]

        archivos_generados = {
            'mascaras': [],
            'overlays': [],
            'recortes': [],
            'alphas': []
        }

        # Guardar original
        ruta_original = dir_imagen / "00_original.png"
        Image.fromarray(imagen_original).save(ruta_original)

        # Procesar cada modelo
        for modelo, datos in datos_modelos.items():
            if nombre_imagen not in datos['imagenes']:
                continue

            img_data = datos['imagenes'][nombre_imagen]

            # Obtener detecciones del JSON
            detecciones = img_data['json_data'].get('detecciones', {})

            # Procesar máscaras NPZ
            for npz_data in img_data['mascaras_npz']:
                mascaras_por_umbral = npz_data['mascaras_por_umbral']

                # Usar primer umbral disponible para visualización
                if not mascaras_por_umbral:
                    continue

                primer_umbral = list(mascaras_por_umbral.keys())[0]
                mascaras = mascaras_por_umbral[primer_umbral]

                # Obtener detecciones del umbral correspondiente
                detecciones_umbral = detecciones.get(primer_umbral, {})
                detecciones_lista = detecciones_umbral.get('detecciones', [])
                confianzas = detecciones_umbral.get('confianza_scores', [])

                for idx, mask_info in enumerate(mascaras):
                    mascara = mask_info['mascara']

                    # Redimensionar máscara si es necesario
                    if mascara.shape != (h_orig, w_orig):
                        mascara = cv2.resize(
                            mascara.astype(np.uint8),
                            (w_orig, h_orig),
                            interpolation=cv2.INTER_NEAREST
                        ).astype(np.float32)

                    mascara_bin = (mascara > 0.5).astype(np.uint8)

                    # Obtener confianza
                    confianza = confianzas[idx] if idx < len(confianzas) else 0.0

                    # Calcular bbox de la máscara
                    contornos, _ = cv2.findContours(
                        mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                    )

                    if len(contornos) == 0:
                        continue

                    contorno_principal = max(contornos, key=cv2.contourArea)
                    x, y, w, h = cv2.boundingRect(contorno_principal)
                    bbox = [x, y, w, h]

                    base_name = f"{modelo}_person_{idx:02d}"

                    # 1. Máscara sola
                    if self.config.GENERAR_MASCARAS_INDIVIDUALES:
                        mask_img = (mascara_bin * 255).astype(np.uint8)
                        ruta_mask = dir_personas / f"{base_name}_mask.png"
                        cv2.imwrite(str(ruta_mask), mask_img)
                        archivos_generados['mascaras'].append(ruta_mask)

                    # 2. Overlay sobre original
                    if self.config.GENERAR_OVERLAYS:
                        overlay = imagen_original.copy()
                        color = self.config.COLORES_MODELOS.get(modelo, (0, 255, 0))
                        overlay[mascara_bin == 1] = color

                        blended = cv2.addWeighted(
                            imagen_original, 1 - self.config.ALPHA_OVERLAY,
                            overlay, self.config.ALPHA_OVERLAY,
                            0
                        )

                        # Añadir bounding box y texto
                        cv2.rectangle(blended, (x, y), (x+w, y+h), color, 2)

                        texto = f"{modelo.upper()} | Conf: {confianza:.2f}"
                        cv2.putText(blended, texto, (x, y-10),
                                  cv2.FONT_HERSHEY_SIMPLEX, 0.5, color, 2)

                        ruta_overlay = dir_personas / f"{base_name}_overlay.png"
                        Image.fromarray(blended).save(ruta_overlay)
                        archivos_generados['overlays'].append(ruta_overlay)

                    # 3. Recorte
                    if self.config.GENERAR_RECORTES:
                        if w > 0 and h > 0:
                            recorte = imagen_original[y:y+h, x:x+w].copy()
                            ruta_recorte = dir_personas / f"{base_name}_cutout.png"
                            Image.fromarray(recorte).save(ruta_recorte)
                            archivos_generados['recortes'].append(ruta_recorte)

                    # 4. Recorte con alpha
                    if self.config.GENERAR_ALPHA:
                        if w > 0 and h > 0:
                            recorte = imagen_original[y:y+h, x:x+w].copy()
                            mask_crop = mascara_bin[y:y+h, x:x+w]

                            rgba = np.zeros((recorte.shape[0], recorte.shape[1], 4), dtype=np.uint8)
                            rgba[:, :, :3] = recorte
                            rgba[:, :, 3] = mask_crop * 255

                            ruta_alpha = dir_personas / f"{base_name}_alpha.png"
                            Image.fromarray(rgba, mode='RGBA').save(ruta_alpha)
                            archivos_generados['alphas'].append(ruta_alpha)

        return archivos_generados

print("✅ Clase GeneradorMascarasIndividualesYOLO definida")

✅ Clase GeneradorMascarasIndividualesYOLO definida


In [6]:
# =============================================================================
# GENERADOR DE COMPARATIVAS
# =============================================================================

class GeneradorComparativasYOLO:
    """Genera visualizaciones comparativas entre modelos YOLOv8"""

    def __init__(self, config: ConfigVisualizadorYOLO):
        self.config = config

    def comparar_modelos_para_imagen(self, nombre_imagen: str,
                                     datos_modelos: Dict,
                                     ruta_imagen_original: Path) -> Optional[Path]:
        """Genera comparativa de todos los modelos para una imagen"""

        # Cargar imagen original
        imagen_original = cv2.imread(str(ruta_imagen_original))
        if imagen_original is None:
            return None

        imagen_original = cv2.cvtColor(imagen_original, cv2.COLOR_BGR2RGB)
        h_orig, w_orig = imagen_original.shape[:2]

        # Filtrar modelos disponibles
        modelos_disponibles = []
        for modelo, datos in datos_modelos.items():
            if nombre_imagen in datos['imagenes']:
                modelos_disponibles.append(modelo)

        if not modelos_disponibles:
            return None

        # Crear figura
        n_modelos = len(modelos_disponibles)
        n_cols = min(n_modelos + 1, 3)
        n_rows = (n_modelos + 1 + n_cols - 1) // n_cols

        fig, axes = plt.subplots(n_rows, n_cols, figsize=(6*n_cols, 6*n_rows))
        if n_rows == 1 and n_cols == 1:
            axes = np.array([[axes]])
        elif n_rows == 1 or n_cols == 1:
            axes = axes.reshape(n_rows, n_cols)

        fig.suptitle(f'Comparativa Modelos YOLOv8-seg - {nombre_imagen}',
                    fontsize=16, fontweight='bold')

        axes_flat = axes.flatten()

        # Mostrar original
        axes_flat[0].imshow(imagen_original)
        axes_flat[0].set_title('Original', fontsize=14, fontweight='bold')
        axes_flat[0].axis('off')

        # Mostrar cada modelo
        for idx, modelo in enumerate(modelos_disponibles, start=1):
            ax = axes_flat[idx]

            img_data = datos_modelos[modelo]['imagenes'][nombre_imagen]
            detecciones = img_data['json_data'].get('detecciones', {})

            # Crear overlay
            overlay = imagen_original.copy()
            color = self.config.COLORES_MODELOS.get(modelo, (0, 255, 0))

            total_personas = 0

            # Procesar máscaras NPZ
            for npz_data in img_data['mascaras_npz']:
                mascaras_por_umbral = npz_data['mascaras_por_umbral']

                if not mascaras_por_umbral:
                    continue

                primer_umbral = list(mascaras_por_umbral.keys())[0]
                mascaras = mascaras_por_umbral[primer_umbral]

                # Obtener detecciones del umbral
                detecciones_umbral = detecciones.get(primer_umbral, {})
                confianzas = detecciones_umbral.get('confianza_scores', [])

                for mask_idx, mask_info in enumerate(mascaras):
                    mascara = mask_info['mascara']

                    # Redimensionar si es necesario
                    if mascara.shape != (h_orig, w_orig):
                        mascara = cv2.resize(
                            mascara.astype(np.uint8),
                            (w_orig, h_orig),
                            interpolation=cv2.INTER_NEAREST
                        ).astype(np.float32)

                    mascara_bin = (mascara > 0.5).astype(np.uint8)
                    overlay[mascara_bin == 1] = color

                    # Calcular bbox
                    contornos, _ = cv2.findContours(
                        mascara_bin, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE
                    )

                    if len(contornos) > 0:
                        contorno = max(contornos, key=cv2.contourArea)
                        x, y, w, h = cv2.boundingRect(contorno)

                        confianza = confianzas[mask_idx] if mask_idx < len(confianzas) else 0.0

                        rect = mpatches.Rectangle(
                            (x, y), w, h,
                            fill=False, edgecolor=np.array(color)/255, linewidth=2
                        )
                        ax.add_patch(rect)

                        ax.text(x, y-5, f'{confianza:.2f}',
                               color=np.array(color)/255, fontsize=10,
                               fontweight='bold',
                               bbox=dict(facecolor='black', alpha=0.7, pad=2))

                        total_personas += 1

            # Blend
            blended = cv2.addWeighted(
                imagen_original, 1 - self.config.ALPHA_OVERLAY,
                overlay, self.config.ALPHA_OVERLAY,
                0
            )

            ax.imshow(blended)

            # Título con nombre del modelo y número de personas
            modelo_display = modelo.replace('yv8', 'YOLOv8-').replace('_seg', '')
            ax.set_title(f'{modelo_display}\n{total_personas} personas',
                        fontsize=12, fontweight='bold')
            ax.axis('off')

        # Ocultar axes sobrantes
        for idx in range(len(modelos_disponibles) + 1, len(axes_flat)):
            axes_flat[idx].axis('off')

        plt.tight_layout()

        # Guardar
        dir_imagen = self.config.OUTPUT_PATH / "por_imagen" / Path(nombre_imagen).stem
        ruta_salida = dir_imagen / "03_comparativa_modelos.png"
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

        return ruta_salida

print("✅ Clase GeneradorComparativasYOLO definida")

✅ Clase GeneradorComparativasYOLO definida


In [7]:
# =============================================================================
# GENERADOR DE ESTADÍSTICAS
# =============================================================================

class GeneradorEstadisticasYOLO:
    """Genera visualizaciones estadísticas de los resultados"""

    def __init__(self, config: ConfigVisualizadorYOLO):
        self.config = config

    def generar_estadisticas_globales(self, datos_modelos: Dict) -> Path:
        """Genera gráficas de estadísticas globales"""

        # Recopilar datos
        estadisticas = {}

        for modelo, datos in datos_modelos.items():
            total_imagenes = len(datos['imagenes'])
            total_personas = 0
            confianzas_totales = []

            for img_data in datos['imagenes'].values():
                detecciones = img_data['json_data'].get('detecciones', {})

                for umbral_key, det_data in detecciones.items():
                    total_personas += det_data.get('num_personas', 0)
                    confianzas_totales.extend(det_data.get('confianza_scores', []))

            estadisticas[modelo] = {
                'total_imagenes': total_imagenes,
                'total_personas': total_personas,
                'personas_promedio': total_personas / total_imagenes if total_imagenes > 0 else 0,
                'confianza_promedio': np.mean(confianzas_totales) if confianzas_totales else 0,
                'confianza_std': np.std(confianzas_totales) if confianzas_totales else 0
            }

        # Crear figura
        fig = plt.figure(figsize=(16, 10))
        gs = GridSpec(2, 2, figure=fig)

        # Gráfica 1: Total de personas detectadas por modelo
        ax1 = fig.add_subplot(gs[0, 0])
        modelos = list(estadisticas.keys())
        personas = [estadisticas[m]['total_personas'] for m in modelos]
        colores_plot = [np.array(self.config.COLORES_MODELOS.get(m, (0, 255, 0)))/255 for m in modelos]

        ax1.bar(modelos, personas, color=colores_plot)
        ax1.set_title('Total de Personas Detectadas por Modelo', fontsize=14, fontweight='bold')
        ax1.set_xlabel('Modelo')
        ax1.set_ylabel('Número de Personas')
        ax1.grid(axis='y', alpha=0.3)

        # Gráfica 2: Promedio de personas por imagen
        ax2 = fig.add_subplot(gs[0, 1])
        promedios = [estadisticas[m]['personas_promedio'] for m in modelos]
        ax2.bar(modelos, promedios, color=colores_plot)
        ax2.set_title('Promedio de Personas por Imagen', fontsize=14, fontweight='bold')
        ax2.set_xlabel('Modelo')
        ax2.set_ylabel('Personas/Imagen')
        ax2.grid(axis='y', alpha=0.3)

        # Gráfica 3: Confianza promedio
        ax3 = fig.add_subplot(gs[1, 0])
        confianzas = [estadisticas[m]['confianza_promedio'] for m in modelos]
        confianzas_std = [estadisticas[m]['confianza_std'] for m in modelos]
        ax3.bar(modelos, confianzas, yerr=confianzas_std, color=colores_plot, capsize=5)
        ax3.set_title('Confianza Promedio por Modelo', fontsize=14, fontweight='bold')
        ax3.set_xlabel('Modelo')
        ax3.set_ylabel('Confianza')
        ax3.set_ylim([0, 1])
        ax3.grid(axis='y', alpha=0.3)

        # Gráfica 4: Tabla resumen
        ax4 = fig.add_subplot(gs[1, 1])
        ax4.axis('tight')
        ax4.axis('off')

        tabla_data = []
        for modelo in modelos:
            stats = estadisticas[modelo]
            tabla_data.append([
                modelo.replace('yv8', 'YOLOv8-').replace('_seg', ''),
                f"{stats['total_imagenes']}",
                f"{stats['total_personas']}",
                f"{stats['personas_promedio']:.2f}",
                f"{stats['confianza_promedio']:.3f}"
            ])

        tabla = ax4.table(
            cellText=tabla_data,
            colLabels=['Modelo', 'Imágenes', 'Personas', 'Pers/Img', 'Conf.'],
            loc='center',
            cellLoc='center'
        )
        tabla.auto_set_font_size(False)
        tabla.set_fontsize(10)
        tabla.scale(1, 2)
        ax4.set_title('Resumen Estadístico', fontsize=14, fontweight='bold', pad=20)

        plt.suptitle('Estadísticas Globales YOLOv8-seg', fontsize=16, fontweight='bold', y=0.98)
        plt.tight_layout()

        # Guardar
        ruta_salida = self.config.OUTPUT_PATH / "estadisticas" / "estadisticas_globales.png"
        plt.savefig(ruta_salida, dpi=150, bbox_inches='tight')
        plt.close()

        return ruta_salida

print("✅ Clase GeneradorEstadisticasYOLO definida")

✅ Clase GeneradorEstadisticasYOLO definida


In [10]:
# =============================================================================
# EJECUCIÓN PRINCIPAL
# =============================================================================

print(f"\n{'='*80}")
print("VISUALIZADOR YOLOV8-SEG - INICIO")
print(f"{'='*80}\n")

# 1. Buscar ejecuciones disponibles
ejecutiones = sorted([d for d in config.RESULTADOS_PATH.glob("ejecucion_*") if d.is_dir()])

if not ejecutiones:
    print("❌ No se encontraron ejecuciones")
else:
    print(f"✅ Ejecuciones encontradas: {len(ejecutiones)}")
    for i, ej in enumerate(ejecutiones, 1):
        print(f"   [{i}] {ej.name}")

    # Seleccionar la más reciente
    directorio_ejecucion = ejecutiones[-1]
    print(f"\n📂 Seleccionada: {directorio_ejecucion.name}")

    # 2. Cargar datos
    cargador = CargadorResultadosYOLO(directorio_ejecucion)
    datos_modelos = cargador.cargar_ejecucion_completa()

    # 3. Obtener lista de imágenes
    nombres_imagenes = cargador.obtener_lista_imagenes()
    rutas_imagenes = {}

    for nombre in nombres_imagenes:
        ruta_img = config.IMAGENES_PATH / nombre
        if ruta_img.exists():
            rutas_imagenes[nombre] = ruta_img
        else:
            print(f"⚠️ Imagen no encontrada: {nombre}")

    print(f"\n📸 Imágenes disponibles: {len(rutas_imagenes)}")
    for nombre in sorted(rutas_imagenes.keys()):
        print(f"   • {nombre}")

    # 4. Generar máscaras individuales
    if config.GENERAR_MASCARAS_INDIVIDUALES or config.GENERAR_OVERLAYS or \
       config.GENERAR_RECORTES or config.GENERAR_ALPHA:

        gen_mascaras = GeneradorMascarasIndividualesYOLO(config)

        print(f"\n{'='*80}")
        print("GENERANDO MÁSCARAS INDIVIDUALES")
        print(f"{'='*80}\n")

        resultados_mascaras = {}

        for nombre_imagen in tqdm(rutas_imagenes.keys(), desc="Procesando imágenes"):
            ruta_img = rutas_imagenes[nombre_imagen]

            archivos = gen_mascaras.generar_para_imagen(
                nombre_imagen, datos_modelos, ruta_img
            )

            resultados_mascaras[nombre_imagen] = archivos

        print(f"\n✅ Máscaras individuales generadas")

    # 5. Generar comparativas
    if config.GENERAR_COMPARATIVAS:
        gen_comparativas = GeneradorComparativasYOLO(config)

        print(f"\n{'='*80}")
        print("GENERANDO COMPARATIVAS")
        print(f"{'='*80}\n")

        for nombre_imagen in tqdm(rutas_imagenes.keys(), desc="Comparativas"):
            ruta_img = rutas_imagenes[nombre_imagen]

            ruta_comp = gen_comparativas.comparar_modelos_para_imagen(
                nombre_imagen, datos_modelos, ruta_img
            )

            if ruta_comp:
                print(f"✅ {nombre_imagen} → {ruta_comp.name}")

        print(f"\n✅ Comparativas generadas")

    # 6. Generar estadísticas
    if config.GENERAR_ESTADISTICAS:
        gen_stats = GeneradorEstadisticasYOLO(config)

        print(f"\n{'='*80}")
        print("GENERANDO ESTADÍSTICAS")
        print(f"{'='*80}\n")

        ruta_stats = gen_stats.generar_estadisticas_globales(datos_modelos)
        print(f"✅ Estadísticas guardadas en: {ruta_stats}")

    # 7. Resumen final
    print(f"\n{'='*80}")
    print("📊 RESUMEN DE RESULTADOS")
    print(f"{'='*80}\n")

    print(f"📁 Directorio de salida: {config.OUTPUT_PATH}")

    if 'resultados_mascaras' in locals():
        print(f"\n🎨 ARCHIVOS GENERADOS:")
        for nombre_imagen, archivos in resultados_mascaras.items():
            print(f"\n📸 {nombre_imagen}:")
            for tipo, lista in archivos.items():
                if lista:
                    print(f"   • {tipo}: {len(lista)} archivos")

    print(f"\n{'='*80}")
    print("✅ PROCESO COMPLETADO")
    print(f"{'='*80}\n")


VISUALIZADOR YOLOV8-SEG - INICIO

✅ Ejecuciones encontradas: 1
   [1] ejecucion_20251009_192747

📂 Seleccionada: ejecucion_20251009_192747

CARGANDO RESULTADOS DE: ejecucion_20251009_192747

📂 Cargando modelo: yv8n_seg
   ✅ 6 imágenes, 82 grupos de máscaras
📂 Cargando modelo: yv8s_seg
   ✅ 6 imágenes, 82 grupos de máscaras
📂 Cargando modelo: yv8m_seg
   ✅ 6 imágenes, 84 grupos de máscaras
📂 Cargando modelo: yv8l_seg
   ✅ 6 imágenes, 84 grupos de máscaras
📂 Cargando modelo: yv8x_seg
   ✅ 6 imágenes, 84 grupos de máscaras

✅ CARGA COMPLETADA
   Modelos cargados: 5


📸 Imágenes disponibles: 6
   • 1.jpg
   • 2.jpg
   • 3.jpg
   • 5.jpg
   • 6.jpg
   • 7.jpg

GENERANDO MÁSCARAS INDIVIDUALES



Procesando imágenes:   0%|          | 0/6 [00:00<?, ?it/s]


✅ Máscaras individuales generadas

GENERANDO COMPARATIVAS



Comparativas:   0%|          | 0/6 [00:00<?, ?it/s]

✅ 1.jpg → 03_comparativa_modelos.png
✅ 2.jpg → 03_comparativa_modelos.png
✅ 3.jpg → 03_comparativa_modelos.png
✅ 5.jpg → 03_comparativa_modelos.png
✅ 6.jpg → 03_comparativa_modelos.png
✅ 7.jpg → 03_comparativa_modelos.png

✅ Comparativas generadas

GENERANDO ESTADÍSTICAS

✅ Estadísticas guardadas en: /content/drive/MyDrive/TFM/yolov8_seg/visualizaciones_avanzadas/estadisticas/estadisticas_globales.png

📊 RESUMEN DE RESULTADOS

📁 Directorio de salida: /content/drive/MyDrive/TFM/yolov8_seg/visualizaciones_avanzadas

🎨 ARCHIVOS GENERADOS:

📸 1.jpg:
   • mascaras: 23 archivos
   • overlays: 23 archivos
   • recortes: 23 archivos
   • alphas: 23 archivos

📸 2.jpg:
   • mascaras: 24 archivos
   • overlays: 24 archivos
   • recortes: 24 archivos
   • alphas: 24 archivos

📸 3.jpg:
   • mascaras: 35 archivos
   • overlays: 35 archivos
   • recortes: 35 archivos
   • alphas: 35 archivos

📸 5.jpg:
   • mascaras: 21 archivos
   • overlays: 21 archivos
   • recortes: 21 archivos
   • alphas: 21 arc